# Classes Desbalanceadas

## Upsampling da minoria

Upsampling é uma técnica que consiste em aumentar a quantidade de amostras da classe minoritária, para que ela fique balanceada com a classe majoritária. Para isso, é feito uma amostragem com reposição, ou seja, amostras da classe minoritária são duplicadas aleatoriamente até que ela fique com a mesma quantidade de amostras da classe majoritária.

In [2]:
from sklearn.utils import resample
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import (
    ensemble,
    impute,
    model_selection,    
    preprocessing,
    tree,
)

In [3]:
url = ("https://hbiostat.org/data/repo/titanic3.csv")
df = pd.read_csv(url)
orig_df = df

In [4]:
def tweak_titanic(df):
    df = df.drop(
        columns=[
            "name",
            "ticket",
            "home.dest",
            "boat",
            "body",
            "cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)
    return df

def get_train_test_X_y(
    df, y_col, size=0.3, std_cols=None
) ->(pd.DataFrame, pd.DataFrame, pd.Series, pd.Series):
    y = df[y_col]
    X = df.drop(columns=y_col)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=size, random_state=42
    )
    cols = X.columns
    num_cols = [
        "pclass",
        "age",
        "sibsp",
        "parch",
        "fare",
    ]
    fi = impute.IterativeImputer()
    fitted = fi.fit_transform(X_train[num_cols])
    X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(num_cols)})
    test_fit = fi.transform(X_test[num_cols])
    X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(num_cols)})
    if std_cols:
        std = preprocessing.StandardScaler()
        fitted = std.fit_transform(X_train[std_cols])
        X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(std_cols)})
        test_fit = std.transform(X_test[std_cols])
        X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(std_cols)})

    return X_train, X_test, y_train, y_test

ti_df = tweak_titanic(orig_df)
std_cols = "pclass,age,sibsp,fare".split(",")

X_train: pd.DataFrame
X_test: pd.DataFrame
y_train: pd.Series
y_test: pd.Series = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)

X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)

X: pd.Series = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

A função `resample` do `sklearn` faz isso de forma simples. Ela recebe como parâmetro o conjunto de dados e retorna um novo conjunto de dados com a classe minoritária balanceada.


In [5]:
mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]

df_upsampled = resample(
    surv_df,
    replace=True,
    n_samples=len(death_df),
    random_state=42,
)
df2 = pd.concat([death_df, df_upsampled])

In [6]:
df2.survived.value_counts()

survived
0    809
1    809
Name: count, dtype: int64

In [7]:
from imblearn.over_sampling import RandomOverSampler

In [8]:
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)
pd.Series(y_ros).value_counts()

survived
0    809
1    809
Name: count, dtype: int64

### Downsampling da maioria

Downsampling é uma técnica que consiste em diminuir a quantidade de amostras da classe majoritária, para que ela fique balanceada com a classe minoritária. Para isso, é feito uma amostragem sem reposição, ou seja, amostras da classe majoritária são removidas aleatoriamente até que ela fique com a mesma quantidade de amostras da classe minoritária.

In [9]:
from sklearn.utils import resample
mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]
df_downsampled = resample(
    death_df,
    replace=False,
    n_samples=len(surv_df),
    random_state=42,
)
df3 = pd.concat([surv_df, df_downsampled])
df3.survived.value_counts()

survived
1    500
0    500
Name: count, dtype: int64

A biblioteca `imbalanced-learn` também implementa diversos algoritmos de downsampling.

`ClusterCentroids` é um algoritmo de downsampling que utiliza o algoritmo de agrupamento K-means para reduzir a quantidade de amostras da classe majoritária. Ele recebe como parâmetro o número de clusters que serão criados e retorna um novo conjunto de dados com a classe majoritária balanceada.

`RandomUnderSampler` é um algoritmo de downsampling que remove aleatoriamente amostras da classe majoritária até que ela fique com a mesma quantidade de amostras da classe minoritária. Ele recebe como parâmetro a estratégia de amostragem que será utilizada para remover as amostras da classe majoritária e retorna um novo conjunto de dados com a classe majoritária balanceada.

`NearMiss` é um algoritmo de downsampling que remove amostras da classe majoritária baseado na distância entre as amostras da classe majoritária e as amostras da classe minoritária. Ele recebe como parâmetro a estratégia de amostragem que será utilizada para remover as amostras da classe majoritária e retorna um novo conjunto de dados com a classe majoritária balanceada.

`TomekLinks` reduz as amostras removendo os mais próximos vizinhos da classe majoritária. Ele recebe como parâmetro o conjunto de dados e retorna um novo conjunto de dados com a classe majoritária balanceada.

`EditedNearestNeighbours` reduz as amostras removendo os vizinhos da classe majoritária que não são bem classificados. Ele recebe como parâmetro o conjunto de dados e retorna um novo conjunto de dados com a classe majoritária balanceada.

`RepeatedEditedNearestNeighbours` reduz as amostras removendo os vizinhos da classe majoritária que não são bem classificados. Ele recebe como parâmetro o conjunto de dados e retorna um novo conjunto de dados com a classe majoritária balanceada. Essa classe chamada `Repeated` faz com que o algoritmo seja executado várias vezes, para que o resultado seja mais preciso.

`AllKNN` reduz as amostras removendo os vizinhos da classe majoritária que não são bem classificados. Ele recebe como parâmetro o conjunto de dados e retorna um novo conjunto de dados com a classe majoritária balanceada. A diferença desse algoritmo para o `EditedNearestNeighbours` é que ele utiliza todos os vizinhos para remover as amostras da classe majoritária.

`CondensedNearestNeighbour` reduz as amostras removendo as amostras da classe majoritária que podem ser bem classificadas pelos vizinhos da classe minoritária. Ele recebe como parâmetro o conjunto de dados e retorna um novo conjunto de dados com a classe majoritária balanceada. Se KNN não fizer uma classificação incorreta, a amostra é adicionada ao conjunto de dados.

`OneSidedSelection` Remove amostras com ruídos

`NeighbourhoodCleaningRule` Usa resultados de `EditedNearestNeighbours` e `CondensedNearestNeighbour` para aplicar KNN

`InstanceHardnessThreshold` Usa um classificador para remover amostras com baixa probabilidade de serem bem classificadas

Todas as classes de downsampling recebem como parâmetro o conjunto de dados e retornam um novo conjunto de dados com a classe majoritária balanceada.

É possivel usar o metodo `fit_sample` para aplicar o algoritmo de downsampling e retornar o conjunto de dados balanceado.